# Project Overview

## OSEMN Pipeline

* O - Obtaining our data
* S - Scrubbing/Cleaning our data
* E - Exploring/Visualizing our data - in EMN_modeling
* M - Modeling our data - in EMN_modeling
* N - Interpreting the data - in EMN_modeling

## Notebook Preparation

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
# resampling packages
from imblearn.under_sampling import RandomUnderSampler


import warnings
warnings.filterwarnings('ignore')

# Obtaining our Data

## About The Data

Our data comes from a variety of sources, all aimed at creating a full view of airport delay through various study metrics involving the airport, aircraft, airline, passengers, and weather.

Our primary dataset is the Bureau of Transportation Statistics' Montly On-Time Report, which for the year of 2019 comprises several million rows of data on every flight flown domestically for the entire year. We use and combine these monthly statistics with a variety of other data sets to gain further insights.

We use 5 informational datasets from the Bureau of Transportation Statistics:
* T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY.csv
* B43_AIRCRAFT_INVENTORY.csv
* AIRPORT_COORDINATES.csv
* CARRIER_DECODE.csv
* P10_EMPLOYEES.csv


2 informational datasets from the National Centers for Environmental Information
* Airport_Weather.csv
* Airport_list.csv

The data sets can be refined at download, so I chose features that I needed when acquiring the data.

Our base data of on-time reporting is feature rich. We have detailed information for EVERY flight taken, including the date, the carrier, the tail number, the origin airport, the destination airport, the time the flight left, the reason for delay if delayed, the length of the flight, and the distance it traveled on the flight. We are interested in the delay and will clean for both general delay and specific delay.

In [2]:
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_01.csv')
df.shape

(583985, 33)

In [3]:
df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,32.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,8,2,9E,N820AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,39.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,9,3,9E,N840AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,4,9E,N8969A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,41.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,1,30,3,UA,N819UA,2024,14683,SAT,"San Antonio, TX",12266,...,61.0,47.0,191.0,1,NaN,NaN,NaN,NaN,NaN,NaN
583981,1,30,3,UA,N37462,2022,14843,SJU,"San Juan, PR",12264,...,234.0,240.0,1571.0,7,NaN,NaN,NaN,NaN,NaN,NaN
583982,1,30,3,UA,N37462,2021,12264,IAD,"Washington, DC",14843,...,226.0,233.0,1571.0,7,NaN,NaN,NaN,NaN,NaN,NaN
583983,1,30,3,UA,N26967,2020,12266,IAH,"Houston, TX",14771,...,252.0,231.0,1635.0,7,11.0,0.0,0.0,0.0,11.0,NaN


In [4]:
df.memory_usage().sum()

154172168

In [5]:
df.dtypes

MONTH                    int64
DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
OP_UNIQUE_CARRIER       object
TAIL_NUM                object
OP_CARRIER_FL_NUM        int64
ORIGIN_AIRPORT_ID        int64
ORIGIN                  object
ORIGIN_CITY_NAME        object
DEST_AIRPORT_ID          int64
DEST                    object
DEST_CITY_NAME          object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY_NEW          float64
DEP_DEL15              float64
DEP_TIME_BLK            object
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY_NEW          float64
ARR_TIME_BLK            object
CANCELLED              float64
CANCELLATION_CODE       object
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
DISTANCE               float64
DISTANCE_GROUP           int64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed:

In [6]:
df.describe()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
count,583985.0,583985.000000,583985.000000,583985.000000,583985.000000,583985.000000,583985.000000,567633.000000,567630.000000,567630.000000,...,583851.000000,565963.000000,583985.000000,583985.000000,105222.000000,105222.000000,105222.000000,105222.000000,105222.000000,0.0
mean,1.0,15.960088,3.835626,2537.869334,12659.701982,12659.470015,1326.266642,1331.957814,13.258226,0.174281,...,143.557401,138.610077,797.742767,3.664861,20.106080,4.428551,18.508392,0.079508,24.950590,NaN
std,0.0,8.987942,1.921899,1821.736145,1519.405493,1519.336466,484.081000,495.404020,47.507390,0.379351,...,73.216303,73.078565,589.999261,2.323890,65.762202,38.535323,41.726392,3.203342,50.851973,NaN
min,1.0,1.000000,1.000000,1.000000,10135.000000,10135.000000,1.000000,1.000000,0.000000,0.000000,...,20.000000,16.000000,31.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,1.0,8.000000,2.000000,979.000000,11292.000000,11292.000000,917.000000,921.000000,0.000000,0.000000,...,90.000000,85.000000,363.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1.0,16.000000,4.000000,2114.000000,12889.000000,12889.000000,1320.000000,1328.000000,0.000000,0.000000,...,125.000000,121.000000,640.000000,3.000000,0.000000,0.000000,5.000000,0.000000,0.000000,NaN
75%,1.0,24.000000,5.000000,3902.000000,13931.000000,13931.000000,1730.000000,1738.000000,5.000000,0.000000,...,175.000000,170.000000,1037.000000,5.000000,16.000000,0.000000,21.000000,0.000000,29.000000,NaN
max,1.0,31.000000,7.000000,7439.000000,16218.000000,16218.000000,2359.000000,2400.000000,1651.000000,1.000000,...,703.000000,737.000000,4983.000000,11.000000,1638.000000,1416.000000,1447.000000,816.000000,1514.000000,NaN


In [7]:
df['ORIGIN_AIRPORT_ID'].nunique()

346

In [8]:
df.isna().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                 2543
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                16352
DEP_DELAY_NEW           16355
DEP_DEL15               16355
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                17061
ARR_DELAY_NEW           18022
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      567259
CRS_ELAPSED_TIME          134
ACTUAL_ELAPSED_TIME     18022
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          478763
WEATHER_DELAY          478763
NAS_DELAY              478763
SECURITY_DELAY         478763
LATE_AIRCRAFT_DELAY    478763
Unnamed: 32            583985
dtype: int

## Merge Data

T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY provides information on how many departures were performed (REV_ACRFT_DEP_PERF_510) and how many passengers were enplaned (REV_PAX_ENP_110) by CARRIER and AIRPORT. We'll use this data to provide metrics for the "busy-ness" of an airport and airline.

In [9]:
passengers = pd.read_csv('data/raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2019.csv')
passengers

,OP_UNIQUE_CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,SERVICE_CLASS,REV_ACRFT_DEP_PERF_510,REV_PAX_ENP_110
0,04Q,Tradewind Aviation,15024,K,10.0,39.0
1,04Q,Tradewind Aviation,14843,K,677.0,3649.0
2,04Q,Tradewind Aviation,10257,V,4.0,6.0
3,04Q,Tradewind Aviation,15323,V,1.0,3.0
4,04Q,Tradewind Aviation,10158,V,1.0,2.0
...,...,...,...,...,...,...
27247,ZW,Air Wisconsin Airlines Corp,11637,K,122.0,4535.0
27248,ZW,Air Wisconsin Airlines Corp,11721,K,143.0,5800.0
27249,ZW,Air Wisconsin Airlines Corp,10469,K,248.0,8901.0
27250,ZW,Air Wisconsin Airlines Corp,12884,K,187.0,7923.0


B43_AIRCRAFT_INVENTORY provides information about specific tail numbers. We want to know the age of an aircraft, and how many passengers it seats.

In [10]:
# Load airplane info so we can get seat count
aircraft = pd.read_csv("data/raw_data/B43_AIRCRAFT_INVENTORY.csv",encoding='latin1')
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True)
aircraft

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


AIRPORT_COORDINATES simply provides specific latitide/longitude for airports. We'll use this as location information.

In [11]:
# load coordinates of airports
coords = pd.read_csv('data/raw_data/AIRPORT_COORDINATES.csv')
coords.drop_duplicates(subset='ORIGIN_AIRPORT_ID', inplace=True)
coords

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,10001,Afognak Lake Airport,58.109444,-152.906667
1,10003,Bear Creek Mining Strip,65.548056,-161.071667
2,10004,Lik Mining Camp,68.083333,-163.166667
3,10005,Little Squaw Airport,67.570000,-148.183889
4,10006,Kizhuyak Bay,57.745278,-152.882778
...,...,...,...,...
18128,16908,Deer Park Airport,47.966944,-117.428611
18129,16909,South Texas International at Edinburg,26.441667,-98.122222
18130,16910,Louisa County Freeman Field,38.009722,-77.970000
18131,16911,Caldwell Industrial,43.641944,-116.635833


CARRIER_DECODE is to get a lookup table for airline codes to match into the main On-Time Reports.

In [12]:
# Load proper names of carriers
names = pd.read_csv("data/raw_data/CARRIER_DECODE.csv")
names.drop_duplicates(inplace=True)
names.drop_duplicates(subset=['OP_UNIQUE_CARRIER'], inplace=True)
names

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
3,20342,Q5,40-Mile Air
4,20342,WRB,40-Mile Air
6,19627,CIQ,A/S Conair
7,19072,AAE,AAA Airlines
...,...,...,...
2702,20379,ZKQ,Zantop International
2706,19771,ZAQ,Zas Airline Of Egypt
2707,21118,37,Zeal 320
2708,22069,ZG,ZIPAIR Tokyo Inc.


P10_EMPLOYEES is so we can determine how many employees a carrier has for Passenger Handling (flight attendants) as well as Ground Service, so that we can determine the employees per passenger.

In [13]:
employees = pd.read_csv('data/raw_data/P10_EMPLOYEES.csv')
employees = employees[['OP_UNIQUE_CARRIER', 'PASS_GEN_SVC_ADMIN', 'PASSENGER_HANDLING']]
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index()
employees

,OP_UNIQUE_CARRIER,PASS_GEN_SVC_ADMIN,PASSENGER_HANDLING
0,0WQ,19,0
1,1BQ,41,0
2,2HQ,24,0
3,3EQ,32,0
4,5V,0,0
5,5X,0,0
6,5Y,273,0
7,8C,37,0
8,9E,1361,0
9,9S,3,0


### Weather Data

Weather Data was acquired on a daily basis for each airport used in the data set. We will ultimately use snowfall, ground snow, precipitation, wind speed, and temperature as features in our data set.

In [14]:
weather_report = pd.read_csv('data/raw_data/airport_weather_2019.csv')
weather_report

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,...,WT08,WT09,WESD,WT10,PSUN,TSUN,SN32,SX32,TOBS,WT11
0,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/1/2019,4.70,NaN,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/2/2019,4.92,NaN,0.57,0.0,0.0,56.0,59.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/3/2019,5.37,NaN,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/4/2019,12.08,NaN,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/5/2019,13.42,NaN,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38670,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-27,6.04,NaN,0.00,NaN,NaN,68.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38671,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-28,5.37,NaN,0.06,NaN,NaN,69.0,74.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38672,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-29,7.61,NaN,0.10,NaN,NaN,70.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38673,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-30,5.82,NaN,0.02,NaN,NaN,68.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We also have a set of the airports and their city names to match up with the weather dataset, so that we can then match this up to our On-Time Dataset

In [16]:
cities = pd.read_csv('data/raw_data/airports_list.csv')
cities

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
1,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
2,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
3,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
93,12264,Washington Dulles International,"Washington, DC","WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US"
94,13851,Will Rogers World,"Oklahoma City, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,12191,William P Hobby,"Houston, TX","HOUSTON WILLIAM P HOBBY AIRPORT, TX US"


Merging our cities reference onto the weather report

In [17]:
weather_merge = pd.merge(cities, weather_report, how='left', on='NAME')
weather_merge

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME,STATION,DATE,AWND,PGTM,PRCP,SNOW,...,WT08,WT09,WESD,WT10,PSUN,TSUN,SN32,SX32,TOBS,WT11
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,2019-01-01,4.70,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,2019-01-02,2.01,NaN,0.39,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,2019-01-03,6.26,NaN,0.44,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,2019-01-04,2.01,NaN,0.13,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,2019-01-05,1.79,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35020,10713,Boise Air Terminal,"Boise, ID","BOISE AIR TERMINAL, ID US",USW00024131,2019-12-27,5.82,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35021,10713,Boise Air Terminal,"Boise, ID","BOISE AIR TERMINAL, ID US",USW00024131,2019-12-28,2.24,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35022,10713,Boise Air Terminal,"Boise, ID","BOISE AIR TERMINAL, ID US",USW00024131,2019-12-29,6.26,NaN,0.04,0.1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35023,10713,Boise Air Terminal,"Boise, ID","BOISE AIR TERMINAL, ID US",USW00024131,2019-12-30,2.46,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Get just the fields that we need
weather = weather_merge[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]

In [19]:
# check for NAN
weather.loc[weather['TMAX'].isna()]

,DATE,PRCP,SNOW,SNWD,TMAX,AWND,ORIGIN_AIRPORT_ID
4786,2/11/2019,0.22,NaN,0.0,NaN,9.62,11298
19976,2019-10-06,0.00,NaN,0.0,NaN,12.30,15919
24068,12/23/2019,0.00,NaN,NaN,NaN,1.57,11066
24807,NaN,NaN,NaN,NaN,NaN,NaN,14843
30085,6/19/2019,NaN,NaN,0.0,NaN,7.61,14635
31953,8/1/2019,0.66,0.0,0.0,NaN,7.61,15304


In [20]:
# Drop empty airport rows 
weather.drop(weather.loc[weather['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)

In [22]:
# fill any isna with mean
weather['TMAX'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather['AWND'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather.fillna(0, inplace=True)

In [23]:
# Check no more nan
weather.isna().sum()

DATE                 0
PRCP                 0
SNOW                 0
SNWD                 0
TMAX                 0
AWND                 0
ORIGIN_AIRPORT_ID    0
dtype: int64

In [24]:
# convert to datetime and get month and day of month to conform with On-Time report
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month
weather['DAY_OF_MONTH'] = pd.DatetimeIndex(weather['DATE']).day
weather

,DATE,PRCP,SNOW,SNWD,TMAX,AWND,ORIGIN_AIRPORT_ID,MONTH,DAY_OF_MONTH
0,2019-01-01,0.00,0.0,0.0,45.0,4.70,12992,1,1
1,2019-01-02,0.39,0.0,0.0,39.0,2.01,12992,1,2
2,2019-01-03,0.44,0.0,0.0,41.0,6.26,12992,1,3
3,2019-01-04,0.13,0.0,0.0,47.0,2.01,12992,1,4
4,2019-01-05,0.00,0.0,0.0,62.0,1.79,12992,1,5
...,...,...,...,...,...,...,...,...,...
35020,2019-12-27,0.00,0.0,0.0,35.0,5.82,10713,12,27
35021,2019-12-28,0.00,0.0,0.0,39.0,2.24,10713,12,28
35022,2019-12-29,0.04,0.1,0.0,32.0,6.26,10713,12,29
35023,2019-12-30,0.00,0.0,0.0,34.0,2.46,10713,12,30


# Scrubbing/Cleaning our Data for OVERALL DELAY

## Cleaning Function

In [25]:
def month_cleanup(monthly_data, aircraft, coords, names, weather, passengers, employees):
    
    '''Function which performs features engineering, data merges and cleanup using one month of On-Time data 
    from Bureau of Transportation Services
    Parameters:
    monthly_data: month of on-time data as downloaded from BTS
    aircraft: Aircraft inventory data from BTS
    coords: Airport coordinates data from BTS
    names: Carrier names based on carrier code from BTS
    weather: Daily weather reported at airports from National Center for Environmental Information
    passengers: Yearly passenger information for carriers and airports from BTS
    employees: Employee statistics for carriers from BTS
    
    returns: cleaned month of On-Time reporting
    '''
    
    # start the timer so we can track how long the cleaning function takes
    start = time.time()
    
    # CLEANING
    # drop rows with no departure time, tail number, or were cancelled
    print("Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.")
    monthly_data.drop(monthly_data.loc[monthly_data['DEP_TIME'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['CANCELLED']==1].index, axis=0, inplace=True)
 
    # FEATURE ENGINEERING - SEGMENT NUMBER
    # List flight segment number for daily flight segments by tracking tail number
    print("Adding Flight Number Sequence - SEGMENT_NUMBER")
    monthly_data["SEGMENT_NUMBER"] = monthly_data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)
    
    # FEATURE ENGINEERING - CONCURRENT FLIGHTS
    # Listing the number of concurrent flights at the airport in the time block 
    print("Adding Concurrent Flights - CONCURRENT_FLIGHTS")
    monthly_data['CONCURRENT_FLIGHTS'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")
 
    # MERGING to get NUMBER_OF_SEATS
    print("Applying seat counts to flights - NUMBER_OF_SEATS")   
    # Merge aircraft info with main frame on tail number - get NUMBER_OF_SEATS 
    monthly_data = pd.merge(monthly_data, aircraft, how="left", on='TAIL_NUM')
    # Fill missing aircraft info with means
    monthly_data['NUMBER_OF_SEATS'].fillna((monthly_data['NUMBER_OF_SEATS'].mean()), inplace=True)
    # simplify data type of number of seats to reduce memory usage
    monthly_data['NUMBER_OF_SEATS'] = monthly_data['NUMBER_OF_SEATS'].astype('int16')

    # MERGING
    # Merge to get proper carrier name
    print("Applying Carrier Names - CARRIER_NAME")  
    monthly_data = pd.merge(monthly_data, names, how='left', on=['OP_UNIQUE_CARRIER'])
    
    # FEATURE ENGINEERING - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
    # Add monthly flight statistics for carrier and airport
    print("Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH")
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    
    # FEATURE ENGINEERING - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
    #Add monthly passenger statistics for carrier and airport
    print("Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE")
    monthly_airport_passengers = pd.DataFrame(passengers.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['AVG_MONTHLY_PASS_AIRPORT'] = (monthly_data['REV_PAX_ENP_110']/12).astype('int64')
    monthly_airline_passengers = pd.DataFrame(passengers.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['AVG_MONTHLY_PASS_AIRLINE'] = (monthly_data['REV_PAX_ENP_110_y']/12).astype('int64')
    
    # MERGING
    # Add employee stats then FEATURE ENGINEER FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
    print("Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS")
    monthly_data = pd.merge(monthly_data, employees, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['FLT_ATTENDANTS_PER_PASS'] = monthly_data['PASSENGER_HANDLING']/monthly_data['REV_PAX_ENP_110_y']
    monthly_data['GROUND_SERV_PER_PASS'] = monthly_data['PASS_GEN_SVC_ADMIN']/monthly_data['REV_PAX_ENP_110_y']
    
    # FEATURE ENGINEERING - PLANE AGE
    # Calculate age of plane
    print("Calculate Fleet Age - PLANE_AGE")
    monthly_data['MANUFACTURE_YEAR'].fillna((monthly_data['MANUFACTURE_YEAR'].mean()), inplace=True)
    monthly_data['PLANE_AGE'] = 2019 - monthly_data['MANUFACTURE_YEAR']

    # MERGING
    # Merge to get airport coordinates
    print("Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT")
    monthly_data = pd.merge(monthly_data, coords, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['LATITUDE'] = round(monthly_data['LATITUDE'], 3)
    monthly_data['LONGITUDE'] = round(monthly_data['LONGITUDE'], 3)

    # FEATURE ENGINEERING - PREVIOUS AIRPORT
    # Get previous airport for tail number
    print("Adding airports - PREVIOUS_AIRPORT")
    segment_temp = monthly_data[['DAY_OF_MONTH', 'TAIL_NUM', 'DISPLAY_AIRPORT_NAME', 'SEGMENT_NUMBER']]
    monthly_data = pd.merge_asof(monthly_data.sort_values('SEGMENT_NUMBER'), segment_temp.sort_values('SEGMENT_NUMBER'), on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], allow_exact_matches=False)
    monthly_data['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    monthly_data.rename(columns={"DISPLAY_AIRPORT_NAME_y": "PREVIOUS_AIRPORT", "DISPLAY_AIRPORT_NAME_x": "DEPARTING_AIRPORT"}, inplace=True)  
    
    # CLEANING  
    # Drop airports below the 10th percentile
    print("Dropping bottom 10% of airports")
    monthly_data.drop(monthly_data.loc[monthly_data['AIRPORT_FLIGHTS_MONTH'] < 1100].index, axis=0, inplace=True)
    
    # MERGING
    # Merge weather data
    print("Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND")
    monthly_data = pd.merge(monthly_data, weather, how='inner', on=['ORIGIN_AIRPORT_ID', 'MONTH', 'DAY_OF_MONTH'])

    
    # CLEANING
    # drop columns that we won't use
    print("Clean up unneeded columns")
    monthly_data.drop(columns = ['ORIGIN',  'DEST',  
                   'CRS_DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 
                   'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME', 'DISTANCE',
                   'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
                  'ARR_DELAY_NEW', 'Unnamed: 32',  'ARR_TIME_BLK', 'ACTUAL_ELAPSED_TIME',
                  'DEST_AIRPORT_ID', 'DEST_CITY_NAME',  'OP_CARRIER_FL_NUM',  'OP_UNIQUE_CARRIER',
                       'AIRLINE_ID', 'DATE', 'DAY_OF_MONTH', 'TAIL_NUM','DEP_TIME',
                    'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',  'PASSENGER_HANDLING', 'REV_PAX_ENP_110_x', 'REV_PAX_ENP_110_y', 
                                 'PASS_GEN_SVC_ADMIN', 'MANUFACTURE_YEAR',
                                 ],
                    axis=1, inplace=True)
    
    # CLEANING
    # specify data types of various fields to reduce memory usage
    print("Cleaning up data types")
    monthly_data['MONTH'] = monthly_data['MONTH'].astype('object')
    monthly_data['DAY_OF_WEEK'] = monthly_data['DAY_OF_WEEK'].astype('object')
    monthly_data['DEP_DEL15'] = monthly_data['DEP_DEL15'].astype('int8')
    monthly_data['DISTANCE_GROUP'] = monthly_data['DISTANCE_GROUP'].astype('int8')
    monthly_data['SEGMENT_NUMBER'] = monthly_data['SEGMENT_NUMBER'].astype('int8')
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data['AIRLINE_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['PLANE_AGE'] = monthly_data['PLANE_AGE'].astype('int32')
    
    # reset index
    monthly_data.reset_index(inplace=True, drop=True)
    
    # print elapsed time
    print(f'Elapsed Time: {time.time() - start}')
    
    print("FINISHED")
    
    # return cleaned file
    return monthly_data

## Process Data with Cleaning Function

In [26]:
# Read and process each month of raw data using the cleaning function

df = pd.read_csv('data/raw_data/ONTIME_REPORTING_01.csv')
month01 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_02.csv')
month02 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_03.csv')
month03 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_04.csv')
month04 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_05.csv')
month05 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_06.csv')
month06 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_07.csv')
month07 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_08.csv')
month08 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_09.csv')
month09 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_10.csv')
month10 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_11.csv')
month11 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_12.csv')
month12 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

# COMBINE MASTER FILE

combined = pd.concat([month01, month02, month03, month04, month05, month06, month07, month08, month09, month10, month11, month12]).reset_index(drop=True)
combined.to_pickle("data/pkl/train_val.pkl")
combined.to_csv('data/train_val.csv', index=False)

Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 8.679861068725586
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding

Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 9.679775953292847
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding empl

## More Cleaning

In [27]:
# Loads in our cleaned file in case we aren't running the entire notebook
all_data = pd.read_pickle("data/pkl/train_val.pkl")

## Data Split - Test and Validation

We need to split our data into a testing and validation set, because we will be using target encoding on our data. It's important that our target encoding not utilize any information found in either our Validation or our Test sets of new, unseen data. In fact, we use the target encoding that is created in our Train set to populate our Val and Test set target encodings, ensuring that our Validation and Test sets contain no leakage.

In [28]:
# Split into subsets
train, validate = train_test_split(all_data, test_size=.3, random_state=42, stratify=all_data['DEP_DEL15'])

# Our Train set
train

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
1052717,3,1,0,0800-0859,9,1,22,176,American Airlines Inc.,7956,...,18,Miami International,25.792,-80.286,NONE,0.00,0.0,0.0,89.0,6.71
2005960,4,1,0,1600-1659,6,1,24,198,United Air Lines Inc.,11588,...,27,Newark Liberty International,40.696,-74.172,NONE,1.00,0.0,0.0,70.0,19.01
649811,2,1,0,1700-1759,1,5,58,90,Comair Inc.,28011,...,3,Atlanta Municipal,33.641,-84.427,Douglas Municipal,0.03,0.0,0.0,56.0,4.92
748602,2,3,0,0700-0759,1,1,19,143,Southwest Airlines Co.,11500,...,21,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,50.0,11.41
5243365,10,1,1,1100-1159,1,2,31,143,Southwest Airlines Co.,14566,...,19,McCarran International,36.080,-115.152,Nashville International,0.00,0.0,0.0,79.0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915565,6,3,0,0700-0759,2,1,20,180,Delta Air Lines Inc.,6007,...,0,Tampa International,27.973,-82.535,NONE,0.06,0.0,0.0,89.0,10.74
1126418,3,6,1,2000-2059,3,7,42,143,Southwest Airlines Co.,33799,...,16,Atlanta Municipal,33.641,-84.427,Louis Armstrong New Orleans International,0.00,0.0,0.0,75.0,9.62
3419407,7,2,0,1500-1559,2,3,31,76,SkyWest Airlines Inc.,10124,...,3,Salt Lake City International,40.784,-111.967,Friedman Memorial,0.00,0.0,0.0,101.0,7.61
785623,2,3,0,1600-1659,1,5,31,50,Endeavor Air Inc.,12355,...,16,LaGuardia,40.779,-73.876,Bangor International,0.47,0.0,0.0,46.0,10.51


In [29]:
# Our Validation set
validate

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
4689157,8,4,0,0900-0959,8,3,62,150,JetBlue Airways,23099,...,13,Stapleton International,39.774,-104.880,John F. Kennedy International,0.46,0.0,0.0,84.0,7.83
2495519,5,2,0,0600-0659,4,1,30,162,JetBlue Airways,11587,...,14,Newark Liberty International,40.696,-74.172,NONE,0.28,0.0,0.0,52.0,6.49
2434006,5,1,0,1300-1359,6,3,5,143,Southwest Airlines Co.,2588,...,14,General Mitchell Field,42.950,-87.897,LaGuardia,0.00,0.0,0.0,54.0,8.28
497013,1,1,0,1800-1859,1,6,5,143,Southwest Airlines Co.,2294,...,12,Hollywood-Burbank Midpoint,34.200,-118.358,McCarran International,0.00,0.0,0.0,36.0,8.95
289368,1,2,0,1900-1959,4,4,24,50,American Eagle Airlines Inc.,7835,...,14,Miami International,25.792,-80.286,Jacksonville International,0.00,0.0,0.0,82.0,10.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720768,8,7,0,2000-2059,2,7,3,50,American Eagle Airlines Inc.,4171,...,15,Cincinnati/Northern Kentucky International,39.048,-84.669,Chicago O'Hare International,0.00,0.0,0.0,89.0,2.46
847371,2,5,0,1500-1559,3,3,45,50,SkyWest Airlines Inc.,16530,...,17,Stapleton International,39.774,-104.880,Wichita Mid-Continent,0.41,6.8,0.0,28.0,8.50
4092377,8,3,1,2100-2159,1,6,25,76,"Midwest Airline, Inc.",10613,...,11,Philadelphia International,39.868,-75.249,Portland International Jetport,0.24,0.0,0.0,92.0,8.95
1767597,4,7,0,0900-0959,2,2,15,143,Southwest Airlines Co.,6763,...,15,Nashville International,36.127,-86.682,Philadelphia International,0.00,0.0,0.0,65.0,5.37


## Target Encode

For our target encoding, we will group monthly delay statistics by the following categories:
* Carrier
* Airport (Use for both departing airport and arriving airport)
* Day of Week
* Departure Block


In [30]:
# Create lookup tables 

carrier_historical = pd.DataFrame(train.groupby(['CARRIER_NAME', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
carrier_historical.rename(columns={'DEP_DEL15':'CARRIER_HISTORICAL'}, inplace=True)

airport_historical = pd.DataFrame(train.groupby(['DEPARTING_AIRPORT', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
airport_historical.rename(columns={'DEP_DEL15':'DEP_AIRPORT_HIST'}, inplace=True)

prev_airport_historical = airport_historical
prev_airport_historical.rename(columns={'DEPARTING_AIRPORT':'PREVIOUS_AIRPORT', 'DEP_DEL15':'PREV_AIRPORT_HIST'}, inplace=True)

day_historical = pd.DataFrame(train.groupby(['DAY_OF_WEEK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
day_historical.rename(columns={'DEP_DEL15':'DAY_HISTORICAL'}, inplace=True)

dep_block_lookup = pd.DataFrame(train.groupby(['DEP_TIME_BLK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
dep_block_lookup.rename(columns={'DEP_DEL15':'DEP_BLOCK_HIST'}, inplace=True)

In [31]:
# Merge lookup tables back onto data frame

train = pd.merge(train, carrier_historical, how='left')
train = pd.merge(train, airport_historical, how='left')
train = pd.merge(train, prev_airport_historical, how='left')
train = pd.merge(train, day_historical, how='left')
train = pd.merge(train, dep_block_lookup, how='left')

In [32]:
# Now merge these lookup tables onto our Validation data frame

validate = pd.merge(validate, carrier_historical, how='left')
validate = pd.merge(validate, airport_historical, how='left')
validate = pd.merge(validate, prev_airport_historical, how='left')
validate = pd.merge(validate, day_historical, how='left')
validate = pd.merge(validate, dep_block_lookup, how='left')

In [33]:
# check isna

train.isna().sum()

MONTH                                  0
DAY_OF_WEEK                            0
DEP_DEL15                              0
DEP_TIME_BLK                           0
DISTANCE_GROUP                         0
SEGMENT_NUMBER                         0
CONCURRENT_FLIGHTS                     0
NUMBER_OF_SEATS                        0
CARRIER_NAME                           0
AIRPORT_FLIGHTS_MONTH                  0
AIRLINE_FLIGHTS_MONTH                  0
AIRLINE_AIRPORT_FLIGHTS_MONTH          0
AVG_MONTHLY_PASS_AIRPORT               0
AVG_MONTHLY_PASS_AIRLINE               0
FLT_ATTENDANTS_PER_PASS                0
GROUND_SERV_PER_PASS                   0
PLANE_AGE                              0
DEPARTING_AIRPORT                      0
LATITUDE                               0
LONGITUDE                              0
PREVIOUS_AIRPORT                       0
PRCP                                   0
SNOW                                   0
SNWD                                   0
TMAX            

In [34]:
# Fillna with mean
train['DEP_AIRPORT_HIST'].fillna(train['DEP_AIRPORT_HIST'].mean(), inplace=True)

# save files to pickle
train.to_pickle("data/pkl/train.pkl")
train.to_csv('data/train.csv', index=False)

In [35]:
# check na
validate.isna().sum()

MONTH                                 0
DAY_OF_WEEK                           0
DEP_DEL15                             0
DEP_TIME_BLK                          0
DISTANCE_GROUP                        0
SEGMENT_NUMBER                        0
CONCURRENT_FLIGHTS                    0
NUMBER_OF_SEATS                       0
CARRIER_NAME                          0
AIRPORT_FLIGHTS_MONTH                 0
AIRLINE_FLIGHTS_MONTH                 0
AIRLINE_AIRPORT_FLIGHTS_MONTH         0
AVG_MONTHLY_PASS_AIRPORT              0
AVG_MONTHLY_PASS_AIRLINE              0
FLT_ATTENDANTS_PER_PASS               0
GROUND_SERV_PER_PASS                  0
PLANE_AGE                             0
DEPARTING_AIRPORT                     0
LATITUDE                              0
LONGITUDE                             0
PREVIOUS_AIRPORT                      0
PRCP                                  0
SNOW                                  0
SNWD                                  0
TMAX                                  0


In [36]:
# fillna with mean
validate['DEP_AIRPORT_HIST'].fillna(validate['DEP_AIRPORT_HIST'].mean(), inplace=True)

# save to pickle
validate.to_pickle("data/pkl/validate.pkl")
validate.to_csv('data/validate.csv', index=False)

# Scrubbing/Cleaning our Data for SPECIFIC DELAY

## Cleaning Function

In [37]:
def month_cleanup2(monthly_data, aircraft, coords, names, weather, passengers, employees):
    
    '''Function which performs features engineering, data merges and cleanup using one month of On-Time data 
    from Bureau of Transportation Services
    Parameters:
    monthly_data: month of on-time data as downloaded from BTS
    aircraft: Aircraft inventory data from BTS
    coords: Airport coordinates data from BTS
    names: Carrier names based on carrier code from BTS
    weather: Daily weather reported at airports from National Center for Environmental Information
    passengers: Yearly passenger information for carriers and airports from BTS
    employees: Employee statistics for carriers from BTS
    
    returns: cleaned file
    '''
    
    # start the timer so we can track how long the cleaning function takes
    start = time.time()
    
    
    # drop rows with no departure time, tail number, or were cancelled
    print("Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.")
    monthly_data.drop(monthly_data.loc[monthly_data['DEP_TIME'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['CANCELLED']==1].index, axis=0, inplace=True)

  
    
    # List flight segment number for daily flight segments by tracking tail number
    print("Adding Flight Number Sequence - SEGMENT_NUMBER")
    monthly_data["SEGMENT_NUMBER"] = monthly_data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)
    
   
    # Listing the concurrent flights at the airport in the time block 
    print("Adding Concurrent Flights - CONCURRENT_FLIGHTS")
    monthly_data['CONCURRENT_FLIGHTS'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")
 
    
    # Getting seat counts for each aircraft
    print("Applying seat counts to flights - NUMBER_OF_SEATS")   
    # Merge aircraft info with main frame on tail number
    monthly_data = pd.merge(monthly_data, aircraft, how="left", on='TAIL_NUM')
    # Fill missing aircraft info with means
    monthly_data['NUMBER_OF_SEATS'].fillna((monthly_data['NUMBER_OF_SEATS'].mean()), inplace=True)
    # simplify data type of number of seats to reduce memory usage
    monthly_data['NUMBER_OF_SEATS'] = monthly_data['NUMBER_OF_SEATS'].astype('int16')

    
    # Merge proper carrier name
    print("Applying Carrier Names - CARRIER_NAME")  
    monthly_data = pd.merge(monthly_data, names, how='left', on=['OP_UNIQUE_CARRIER'])
    
    
    # Add monthly flight statistics for carrier and airport
    print("Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH")
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    
    
    #Add monthly passenger statistics for carrier and airport
    print("Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE")
    monthly_airport_passengers = pd.DataFrame(passengers.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['AVG_MONTHLY_PASS_AIRPORT'] = (monthly_data['REV_PAX_ENP_110']/12).astype('int64')
    monthly_airline_passengers = pd.DataFrame(passengers.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['AVG_MONTHLY_PASS_AIRLINE'] = (monthly_data['REV_PAX_ENP_110_y']/12).astype('int64')
    
    
    # Add employee stats
    print("Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS")
    monthly_data = pd.merge(monthly_data, employees, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['FLT_ATTENDANTS_PER_PASS'] = monthly_data['PASSENGER_HANDLING']/monthly_data['REV_PAX_ENP_110_y']
    monthly_data['GROUND_SERV_PER_PASS'] = monthly_data['PASS_GEN_SVC_ADMIN']/monthly_data['REV_PAX_ENP_110_y']
    
    
    # Calculate age of plane
    print("Calculate Fleet Age - PLANE_AGE")
    monthly_data['MANUFACTURE_YEAR'].fillna((monthly_data['MANUFACTURE_YEAR'].mean()), inplace=True)
    monthly_data['PLANE_AGE'] = 2019 - monthly_data['MANUFACTURE_YEAR']
    #print(f'Elapsed Time: {time.time() - start}') 

    
    # Merge airport coordinates
    print("Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT")
    monthly_data = pd.merge(monthly_data, coords, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['LATITUDE'] = round(monthly_data['LATITUDE'], 3)
    monthly_data['LONGITUDE'] = round(monthly_data['LONGITUDE'], 3)

    
    # Get previous airport for tail number
    print("Adding airports - PREVIOUS_AIRPORT")
    segment_temp = monthly_data[['DAY_OF_MONTH', 'TAIL_NUM', 'DISPLAY_AIRPORT_NAME', 'SEGMENT_NUMBER']]
    monthly_data = pd.merge_asof(monthly_data.sort_values('SEGMENT_NUMBER'), segment_temp.sort_values('SEGMENT_NUMBER'), on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], allow_exact_matches=False)
    monthly_data['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    monthly_data.rename(columns={"DISPLAY_AIRPORT_NAME_y": "PREVIOUS_AIRPORT", "DISPLAY_AIRPORT_NAME_x": "DEPARTING_AIRPORT"}, inplace=True)  
    
    # Drop airports below the 10th percentile
    print("Dropping bottom 10% of airports")
    monthly_data.drop(monthly_data.loc[monthly_data['AIRPORT_FLIGHTS_MONTH'] < 1100].index, axis=0, inplace=True)
    
    
    # Merge weather data
    print("Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND")
    monthly_data = pd.merge(monthly_data, weather, how='inner', on=['ORIGIN_AIRPORT_ID', 'MONTH', 'DAY_OF_MONTH'])
    
    
    
    # drop columns that we won't use
    print("Clean up unneeded columns")
    monthly_data.drop(columns = ['ORIGIN',  'DEST',  
                   'CRS_DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 
                   'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME', 'DISTANCE',
                  'ARR_DELAY_NEW', 'Unnamed: 32',  'ARR_TIME_BLK', 'ACTUAL_ELAPSED_TIME',
                  'DEST_AIRPORT_ID', 'DEST_CITY_NAME',  'OP_CARRIER_FL_NUM',  'OP_UNIQUE_CARRIER',
                       'AIRLINE_ID', 'DATE', 'DAY_OF_MONTH', 'TAIL_NUM','DEP_TIME',
                    'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',  'PASSENGER_HANDLING', 'REV_PAX_ENP_110_x', 'REV_PAX_ENP_110_y', 
                                 'PASS_GEN_SVC_ADMIN', 'MANUFACTURE_YEAR',
                                 ],
                    axis=1, inplace=True) #,   'DEP_TIME_BLK', 
    
    
    # specify data types of various fields to reduce memory usage
    print("Cleaning up data types")
    monthly_data['MONTH'] = monthly_data['MONTH'].astype('object')
    monthly_data['DAY_OF_WEEK'] = monthly_data['DAY_OF_WEEK'].astype('object')
    monthly_data['DEP_DEL15'] = monthly_data['DEP_DEL15'].astype('int8')
    monthly_data['DISTANCE_GROUP'] = monthly_data['DISTANCE_GROUP'].astype('int8')
    monthly_data['SEGMENT_NUMBER'] = monthly_data['SEGMENT_NUMBER'].astype('int8')
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data['AIRLINE_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['PLANE_AGE'] = monthly_data['PLANE_AGE'].astype('int32')

    monthly_data.reset_index(inplace=True, drop=True)
    
    print(f'Elapsed Time: {time.time() - start}')
    
    print("FINISHED")
    return monthly_data

## Process Data with Cleaning Function

In [38]:
# load and clean all data with cleaning function, then combine

df = pd.read_csv('data/raw_data/ONTIME_REPORTING_01.csv')
month01 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_02.csv')
month02 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_03.csv')
month03 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_04.csv')
month04 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_05.csv')
month05 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_06.csv')
month06 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_07.csv')
month07 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_08.csv')
month08 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_09.csv')
month09 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_10.csv')
month10 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_11.csv')
month11 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_12.csv')
month12 = month_cleanup2(df, aircraft, coords, names, weather, passengers, employees)

# COMBINE MASTER FILE

combined = pd.concat([month01, month02, month03, month04, month05, month06, month07, month08, month09, month10, month11, month12]).reset_index(drop=True)


Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 8.946110248565674
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding

Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 9.67376184463501
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding emplo

In [39]:
# GEt delays only so we can make our data set to look for delay reason

delays_only = combined.loc[combined['DEP_DEL15'] == 1]

delays_only

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
7,1,7,1,0001-0559,7,22.0,0.0,0.0,0.0,0.0,...,3,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91
10,1,7,1,2300-2359,6,28.0,0.0,0.0,0.0,21.0,...,3,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91
15,1,7,1,0700-0759,4,62.0,0.0,9.0,0.0,0.0,...,4,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91
24,1,7,1,1000-1059,3,336.0,0.0,4.0,0.0,863.0,...,19,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91
36,1,7,1,2200-2259,9,19.0,0.0,0.0,0.0,125.0,...,15,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538622,12,7,1,1300-1359,10,6.0,0.0,15.0,0.0,130.0,...,5,Kahului Airport,20.901,-156.434,Los Angeles International,0.00,0.0,0.0,84.0,10.29
6538624,12,7,1,1400-1459,10,0.0,0.0,0.0,0.0,15.0,...,1,Kahului Airport,20.901,-156.434,Portland International,0.00,0.0,0.0,84.0,10.29
6538653,12,7,1,1700-1759,1,3.0,0.0,0.0,0.0,26.0,...,18,Kahului Airport,20.901,-156.434,Honolulu International,0.00,0.0,0.0,84.0,10.29
6538673,12,7,1,1500-1559,1,1.0,0.0,0.0,0.0,18.0,...,18,Kahului Airport,20.901,-156.434,Keahole,0.00,0.0,0.0,84.0,10.29


In [40]:
# check isna
delays_only.isna().sum()

MONTH                                 0
DAY_OF_WEEK                           0
DEP_DEL15                             0
DEP_TIME_BLK                          0
DISTANCE_GROUP                        0
CARRIER_DELAY                    258425
WEATHER_DELAY                    258425
NAS_DELAY                        258425
SECURITY_DELAY                   258425
LATE_AIRCRAFT_DELAY              258425
SEGMENT_NUMBER                        0
CONCURRENT_FLIGHTS                    0
NUMBER_OF_SEATS                       0
CARRIER_NAME                          0
AIRPORT_FLIGHTS_MONTH                 0
AIRLINE_FLIGHTS_MONTH                 0
AIRLINE_AIRPORT_FLIGHTS_MONTH         0
AVG_MONTHLY_PASS_AIRPORT              0
AVG_MONTHLY_PASS_AIRLINE              0
FLT_ATTENDANTS_PER_PASS               0
GROUND_SERV_PER_PASS                  0
PLANE_AGE                             0
DEPARTING_AIRPORT                     0
LATITUDE                              0
LONGITUDE                             0


In [41]:
# drop isna, we have plenty of data
delays_only.dropna(axis=0, inplace=True)
delays_only.reset_index(inplace=True, drop=True)

In [42]:
def check_reason(line):
    '''Check the reason for the delay, if the reason is over .65 of the total delay,
    attribute to single reason. Otherwise attribute to MIXED reason.
    
    parameters: single line of DF
    returns: delay reason'''
    
    carrier = float(line['CARRIER_DELAY'])
    weather = float(line['WEATHER_DELAY'])
    nas = float(line['NAS_DELAY'])
    security = float(line['SECURITY_DELAY'])
    late = float(line['LATE_AIRCRAFT_DELAY'])

    total = carrier + weather + nas + security + late
    
    if carrier/total > .65:
        return 'CARRIER'
    elif weather/total > .65:
        return 'WEATHER'
    elif nas/total > .65:
        return 'NAS'
    elif security/total > .65:
        return 'SECURITY'
    elif late/total > .65:
        return 'LATE_AIRCRAFT'
    else:
        return 'MIXED'

In [43]:
# apply delay reason function to put delay reason in a single column
delays_only['DELAY_REASON'] = delays_only.apply(lambda x: check_reason(x), axis=1)

In [44]:
# drop original delay columns
delays_only.drop(['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1, inplace=True)

In [45]:
delays_only

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,DELAY_REASON
0,1,7,1,0001-0559,7,1,10,186,Frontier Airlines Inc.,13056,...,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91,CARRIER
1,1,7,1,2300-2359,6,1,17,180,Frontier Airlines Inc.,13056,...,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91,MIXED
2,1,7,1,0700-0759,4,1,29,181,Alaska Airlines Inc.,13056,...,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91,CARRIER
3,1,7,1,1000-1059,3,1,29,142,United Air Lines Inc.,13056,...,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91,LATE_AIRCRAFT
4,1,7,1,2200-2259,9,1,9,162,JetBlue Airways,13056,...,McCarran International,36.080,-115.152,NONE,0.00,0.0,0.0,65.0,2.91,LATE_AIRCRAFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015515,12,7,1,1300-1359,10,2,8,181,American Airlines Inc.,2484,...,Kahului Airport,20.901,-156.434,Los Angeles International,0.00,0.0,0.0,84.0,10.29,LATE_AIRCRAFT
1015516,12,7,1,1400-1459,10,2,7,189,Hawaiian Airlines Inc.,2484,...,Kahului Airport,20.901,-156.434,Portland International,0.00,0.0,0.0,84.0,10.29,LATE_AIRCRAFT
1015517,12,7,1,1700-1759,1,4,4,123,Hawaiian Airlines Inc.,2484,...,Kahului Airport,20.901,-156.434,Honolulu International,0.00,0.0,0.0,84.0,10.29,LATE_AIRCRAFT
1015518,12,7,1,1500-1559,1,9,8,123,Hawaiian Airlines Inc.,2484,...,Kahului Airport,20.901,-156.434,Keahole,0.00,0.0,0.0,84.0,10.29,LATE_AIRCRAFT


## Data Split - Test and Validation

We need to split our data into a testing and validation set, because we will be using target encoding on our data. It's important that our target encoding not utilize any information found in either our Validation or our Test sets of new, unseen data. In fact, we use the target encoding that is created in our Train set to populate our Val and Test set target encodings, ensuring that our Validation and Test sets contain no leakage.

In [46]:
# Split into subsets
train, validate = train_test_split(delays_only, test_size=.3, random_state=42)

## Target Encode

For our target encoding, we will group monthly delay statistics by the following categories:
* Carrier
* Airport (Use for both departing airport and arriving airport)
* Day of Week
* Departure Block


In [48]:
# Merge lookup tables back onto data frame

#train = pd.merge(train, delay_type, how='left', left_on='DELAY_REASON', right_on = 'index')
train = pd.merge(train, carrier_historical, how='left')
train = pd.merge(train, airport_historical, how='left')
train = pd.merge(train, prev_airport_historical, how='left')
train = pd.merge(train, day_historical, how='left')
train = pd.merge(train, dep_block_lookup, how='left')


In [49]:
# Now merge these lookup tables onto our Validation data frame

#validate = pd.merge(validate, delay_type, how='left', left_on='DELAY_REASON', right_on = 'index')
validate = pd.merge(validate, carrier_historical, how='left')
validate = pd.merge(validate, airport_historical, how='left')
validate = pd.merge(validate, prev_airport_historical, how='left')
validate = pd.merge(validate, day_historical, how='left')
validate = pd.merge(validate, dep_block_lookup, how='left')

In [50]:
# fillna for historical airport

train['DEP_AIRPORT_HIST'].fillna(train['DEP_AIRPORT_HIST'].mean(), inplace=True)

In [51]:
# drop original delay binary
train.drop('DEP_DEL15', axis=1, inplace=True)


In [52]:
# save files to pickle
train.to_pickle("data/pkl/train_delays.pkl")
train.to_csv('data/train_delays.csv', index=False)

In [53]:
train

,MONTH,DAY_OF_WEEK,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,...,PRCP,SNOW,SNWD,TMAX,AWND,DELAY_REASON,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST
0,4,7,2100-2159,3,1,33,50,American Eagle Airlines Inc.,7011,25138,...,0.00,0.0,0.0,86.0,7.16,CARRIER,0.146509,0.204697,0.203729,0.264607
1,3,4,2200-2259,1,5,11,100,JetBlue Airways,11110,25502,...,0.00,0.0,0.0,57.0,11.18,MIXED,0.249290,0.165380,0.179152,0.194965
2,5,4,1800-1859,2,5,15,143,Southwest Airlines Co.,5562,113709,...,0.00,0.0,0.0,84.0,8.05,LATE_AIRCRAFT,0.248016,0.239330,0.250504,0.309200
3,3,1,1700-1759,3,5,28,142,United Air Lines Inc.,11006,53007,...,0.16,0.0,0.0,60.0,9.17,NAS,0.184016,0.180546,0.161062,0.224798
4,8,1,1800-1859,6,8,10,143,Southwest Airlines Co.,4742,114987,...,1.40,0.0,0.0,78.0,6.04,LATE_AIRCRAFT,0.192933,0.242930,0.181530,0.309009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710859,4,1,2000-2059,1,8,23,100,JetBlue Airways,12669,24966,...,0.31,0.0,0.0,54.0,11.63,LATE_AIRCRAFT,0.266494,0.149187,0.238144,0.247386
710860,5,6,1100-1159,5,3,20,181,American Airlines Inc.,8145,78894,...,0.04,0.0,0.0,88.0,10.96,LATE_AIRCRAFT,0.256803,0.286236,0.179506,0.163243
710861,2,4,1200-1259,6,2,50,76,Mesa Airlines Inc.,13272,15953,...,0.02,0.0,0.0,84.0,7.16,CARRIER,0.208413,0.145307,0.244909,0.228026
710862,8,4,1800-1859,1,7,3,50,Comair Inc.,1181,24496,...,0.00,0.0,0.0,89.0,4.92,LATE_AIRCRAFT,0.232407,0.209670,0.241347,0.309009


In [54]:
# check validation isna
validate.isna().sum()

MONTH                                0
DAY_OF_WEEK                          0
DEP_DEL15                            0
DEP_TIME_BLK                         0
DISTANCE_GROUP                       0
SEGMENT_NUMBER                       0
CONCURRENT_FLIGHTS                   0
NUMBER_OF_SEATS                      0
CARRIER_NAME                         0
AIRPORT_FLIGHTS_MONTH                0
AIRLINE_FLIGHTS_MONTH                0
AIRLINE_AIRPORT_FLIGHTS_MONTH        0
AVG_MONTHLY_PASS_AIRPORT             0
AVG_MONTHLY_PASS_AIRLINE             0
FLT_ATTENDANTS_PER_PASS              0
GROUND_SERV_PER_PASS                 0
PLANE_AGE                            0
DEPARTING_AIRPORT                    0
LATITUDE                             0
LONGITUDE                            0
PREVIOUS_AIRPORT                     0
PRCP                                 0
SNOW                                 0
SNWD                                 0
TMAX                                 0
AWND                     

In [55]:
# fill na with means
validate['DEP_AIRPORT_HIST'].fillna(validate['DEP_AIRPORT_HIST'].mean(), inplace=True)

In [56]:
# drop binary delay
validate.drop('DEP_DEL15', axis=1, inplace=True)


In [57]:
# save files to pickle
validate.to_pickle("data/pkl/validate_delays.pkl")
validate.to_csv('data/validate_delays.csv', index=False)

# Process Test Set

In [58]:
# get weather reports for test set dates

weather_report2 = pd.read_csv('data/raw_data/airport_weather_2020.csv')
weather_merge2 = pd.merge(cities, weather_report2, how='left', on='NAME')
weather2 = weather_merge2[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]
weather2.drop(weather2.loc[weather2['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)
weather2['TMAX'].fillna(round(weather2.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather2['AWND'].fillna(round(weather2.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather2.fillna(0, inplace=True)
weather2['DATE'] = pd.to_datetime(weather2['DATE'])
weather2['MONTH'] = pd.DatetimeIndex(weather2['DATE']).month
weather2['DAY_OF_MONTH'] = pd.DatetimeIndex(weather2['DATE']).day
weather2

,DATE,PRCP,SNOW,SNWD,TMAX,AWND,ORIGIN_AIRPORT_ID,MONTH,DAY_OF_MONTH
0,2020-01-01,0.00,0.0,0.0,50.0,4.92,12992,1,1
1,2020-01-02,0.50,0.0,0.0,53.0,6.04,12992,1,2
2,2020-01-03,0.18,0.0,0.0,56.0,4.03,12992,1,3
3,2020-01-04,0.00,0.0,0.0,50.0,4.03,12992,1,4
4,2020-01-05,0.00,0.0,0.0,65.0,2.91,12992,1,5
...,...,...,...,...,...,...,...,...,...
8370,2020-03-27,0.00,0.0,0.0,50.0,6.04,10713,3,27
8371,2020-03-28,0.00,0.0,0.0,57.0,4.70,10713,3,28
8372,2020-03-29,0.10,0.0,0.0,55.0,7.38,10713,3,29
8373,2020-03-30,0.03,0.0,0.0,54.0,11.41,10713,3,30


In [59]:
# get passenger reports for test set dates
passengers2 = pd.read_csv('data/raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2020.csv')
passengers2

,OP_UNIQUE_CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,SERVICE_CLASS,REV_ACRFT_DEP_PERF_510,REV_PAX_ENP_110
0,04Q,Tradewind Aviation,13535,K,20,105.0
1,04Q,Tradewind Aviation,15024,K,9,38.0
2,04Q,Tradewind Aviation,13987,K,1,2.0
3,04Q,Tradewind Aviation,14843,K,626,3553.0
4,04Q,Tradewind Aviation,12197,K,21,109.0
...,...,...,...,...,...,...
6253,ZW,Air Wisconsin Airlines Corp,11721,K,119,4463.0
6254,ZW,Air Wisconsin Airlines Corp,10469,K,160,5095.0
6255,ZW,Air Wisconsin Airlines Corp,12884,K,159,5165.0
6256,ZW,Air Wisconsin Airlines Corp,15380,K,118,4011.0


## Overall Delay

In [60]:
# process test set with cleanup function for overall delay
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_2020_01.csv')
test01 = month_cleanup(df, aircraft, coords, names, weather2, passengers2, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_2020_02.csv')
test02 = month_cleanup(df, aircraft, coords, names, weather2, passengers2, employees)

# COMBINE FILE
test = pd.concat([test01, test02])

Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 9.173316955566406
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding

In [61]:
# Merge lookup tables back onto data frame

test = pd.merge(test, carrier_historical, how='left')
test = pd.merge(test, airport_historical, how='left')
test = pd.merge(test, prev_airport_historical, how='left')
test = pd.merge(test, day_historical, how='left')
test = pd.merge(test, dep_block_lookup, how='left')

In [62]:
# fill na
test['DEP_AIRPORT_HIST'].fillna(test['DEP_AIRPORT_HIST'].mean(), inplace=True)

# save as pickle
test.to_pickle("data/pkl/test.pkl")
test.to_csv('data/test.csv', index=False)

## Specific Delay

In [63]:
# clean test sets for specific delay
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_2020_01.csv')
test01 = month_cleanup2(df, aircraft, coords, names, weather2, passengers2, employees)
df = pd.read_csv('data/raw_data/ONTIME_REPORTING_2020_02.csv')
test02 = month_cleanup2(df, aircraft, coords, names, weather2, passengers2, employees)


# COMBINE FILE
test = pd.concat([test01, test02])

Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND
Clean up unneeded columns
Cleaning up data types
Elapsed Time: 9.125272989273071
FINISHED
Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding

In [64]:
# drop any missing
test.dropna(axis=0, inplace=True)
test.reset_index(inplace=True, drop=True)

In [65]:
# apply reason function to new column
test['DELAY_REASON'] = test.apply(lambda x: check_reason(x), axis=1)

In [66]:
# drop original delay columns
test.drop(['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1, inplace=True)

In [67]:
# Merge lookup tables back onto data frame

#test = pd.merge(test, delay_type, how='left', left_on='DELAY_REASON', right_on = 'index')
test = pd.merge(test, carrier_historical, how='left')
test = pd.merge(test, airport_historical, how='left')
test = pd.merge(test, prev_airport_historical, how='left')
test = pd.merge(test, day_historical, how='left')
test = pd.merge(test, dep_block_lookup, how='left')

In [68]:
# drop original binary
test.drop('DEP_DEL15', axis=1, inplace=True)


In [69]:
# fill na with mean
test['DEP_AIRPORT_HIST'].fillna(test['DEP_AIRPORT_HIST'].mean(), inplace=True)

# save as pickle
test.to_pickle("data/pkl/test_delays.pkl")
test.to_csv('data/test_delays.csv', index=False)